In [1]:
import os
import librosa
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings(action='ignore')

path='./music/'
file_list=os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.wav')]

music_df=pd.DataFrame(columns=('아티스트','제목','곡의 빠르기','잡음','진폭의 변화','주파수의 밝기','파워'))
count=0
for i in file_list_py:
#     print(i)
    y,sr = librosa.load(path+i)
    tempo = librosa.beat.beat_track(y,sr=sr)
#     print(f'tempo : ',tempo[0])

    zero_crossings=librosa.zero_crossings(y[0:len(y)] ,pad=False)
#     print(f'zero crossings : ',sum(zero_crossings)) # 보완 필요 카운트 방법 고민
    zero_crossings=sum(zero_crossings)

    rolloff=librosa.feature.spectral_rolloff(y,sr=sr)[0]
#     print(f'rolloff : ',np.mean(rolloff))
    rolloff=np.mean(rolloff)

    cent = librosa.feature.spectral_centroid(y, sr=sr)
#     print(f'centroid : ',np.mean(cent))
    cent = np.mean(cent)

    rms=librosa.feature.rms(y=y)[0]
#     print(f'rms : ',np.mean(rms))
    rms=np.mean(rms)

    name = i.split(' - ')
    artist=name[0]
    title=name[1]
    title=re.sub('.wav','',title)
    music_df.loc[count] = [artist,title,tempo[0],zero_crossings,rolloff,cent,rms]
    count += 1

display(music_df)
music_df.to_csv('music_all.csv')

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: './music/'

In [3]:
pd.read_csv('music_all.csv')

,Unnamed: 0,아티스트,제목,곡의 빠르기,잡음,진폭의 변화,주파수의 밝기,파워
0,0,(여자)아이들,주세요,123.046875,408480,4450.751972,2145.944160,0.340491
1,1,10cm,그러나,172.265625,369168,4796.969876,2088.306644,0.212760
2,2,2NE1,Fire,123.046875,542012,6079.712957,2738.215065,0.246861
3,3,2NE1,살아 봤으면 해,184.570312,509662,5021.371112,2468.225559,0.247141
4,4,4minute,거울아 거울아,129.199219,571112,6847.045294,2970.318619,0.275822
...,...,...,...,...,...,...,...,...
332,332,혁오(HYUKOH),위잉위잉,123.046875,298494,3452.353045,1677.857052,0.234736
333,333,화사,LMM,135.999178,419382,3336.793526,1645.248935,0.192747
334,334,황규영,나는 문제없어,123.046875,840950,7476.440230,3466.510580,0.284272
335,335,휘성,일년이면,129.199219,500886,4916.635590,2231.526337,0.180134


In [2]:
import scipy.stats as stats
import pandas as pd

df=pd.read_csv('music_all.csv')
df=df.drop('Unnamed: 0',axis=1)
music_df=df
tempo=stats.norm(np.mean(music_df['곡의 빠르기']),np.std(music_df['곡의 빠르기'])).cdf(music_df['곡의 빠르기'])
rms=stats.norm(np.mean(music_df['파워']),np.std(music_df['파워'])).cdf(df['파워'])
zerocross=stats.norm(np.mean(music_df['잡음']),np.std(music_df['잡음'])).cdf(df['잡음'])
rolloff=stats.norm(np.mean(music_df['진폭의 변화']),np.std(music_df['진폭의 변화'])).cdf(music_df['진폭의 변화'])
centroid=stats.norm(np.mean(music_df['주파수의 밝기']),np.std(music_df['주파수의 밝기'])).cdf(music_df['주파수의 밝기'])
music_df['곡의 빠르기']=tempo
music_df['파워']=rms
music_df['잡음']=zerocross
music_df['진폭의 변화']=rolloff
music_df['주파수의 밝기']=centroid
# 가중치 설정후 X=행복도  Y=흥분도
df_X=music_df['곡의 빠르기']*20+music_df['잡음']*40+music_df['진폭의 변화']*20+music_df['주파수의 밝기']*10+music_df['파워']*10
df_Y=music_df['곡의 빠르기']*40+music_df['잡음']*10+music_df['진폭의 변화']*20+music_df['주파수의 밝기']*20+music_df['파워']*10
music_df['X']=df_X
music_df['Y']=df_Y
df.to_csv("music_final.csv", index=False)
music_df

FileNotFoundError: [Errno 2] No such file or directory: 'music_all.csv'

In [23]:
music_df = pd.read_csv('music_final.csv')
music_df

,아티스트,제목,곡의 빠르기,잡음,진폭의 변화,주파수의 밝기,파워,X,Y
0,(여자)아이들,주세요,0.568162,0.213447,0.214522,0.250805,0.967039,36.370002,43.837891
1,10cm,그러나,0.986721,0.134695,0.330918,0.210676,0.274170,36.589059,54.389390
2,2NE1,Fire,0.568162,0.601910,0.808439,0.753546,0.520322,64.347089,65.188497
3,2NE1,살아 봤으면 해,0.996829,0.501291,0.417501,0.526629,0.522452,58.829030,68.993185
4,4minute,거울아 거울아,0.665485,0.687129,0.951090,0.888338,0.727081,75.970843,77.550069
...,...,...,...,...,...,...,...,...,...
332,혁오(HYUKOH),위잉위잉,0.568162,0.048288,0.035142,0.040497,0.428349,18.686053,29.005632
333,화사,LMM,0.761207,0.239298,0.026924,0.034403,0.162945,27.308003,35.697234
334,황규영,나는 문제없어,0.568162,0.995542,0.989221,0.990742,0.777979,88.656534,80.060952
335,휘성,일년이면,0.665485,0.473706,0.376300,0.317116,0.110613,44.061213,46.330908


In [26]:
music_df['감정'] = True
for i in range(0,len(music_df)):
    x=music_df['X'][i]
    y=music_df['Y'][i]
    if x<50 and y<50:
        if x<y:
            music_df['감정'][i]='울고싶은'
        else:
            music_df['감정'][i]='우울한'
    elif x<50 and y>50:
        if x<(y-50):
            music_df['감정'][i]='무서운'
        else:
            music_df['감정'][i]='긴장되는'
    elif x>50 and y>50:
        if x<y:
            music_df['감정'][i]='경쾌한'
        else:
            music_df['감정'][i]='행복한'
    elif x>50 and y<50:
        if (x-50)<y:
            music_df['감정'][i]='편안한'
        else:
            music_df['감정'][i]='잔잔한'
music_df

,아티스트,제목,곡의 빠르기,잡음,진폭의 변화,주파수의 밝기,파워,X,Y,감성,감정
0,(여자)아이들,주세요,0.568162,0.213447,0.214522,0.250805,0.967039,36.370002,43.837891,True,울고싶은
1,10cm,그러나,0.986721,0.134695,0.330918,0.210676,0.274170,36.589059,54.389390,True,긴장되는
2,2NE1,Fire,0.568162,0.601910,0.808439,0.753546,0.520322,64.347089,65.188497,True,경쾌한
3,2NE1,살아 봤으면 해,0.996829,0.501291,0.417501,0.526629,0.522452,58.829030,68.993185,True,경쾌한
4,4minute,거울아 거울아,0.665485,0.687129,0.951090,0.888338,0.727081,75.970843,77.550069,True,경쾌한
...,...,...,...,...,...,...,...,...,...,...,...
332,혁오(HYUKOH),위잉위잉,0.568162,0.048288,0.035142,0.040497,0.428349,18.686053,29.005632,True,울고싶은
333,화사,LMM,0.761207,0.239298,0.026924,0.034403,0.162945,27.308003,35.697234,True,울고싶은
334,황규영,나는 문제없어,0.568162,0.995542,0.989221,0.990742,0.777979,88.656534,80.060952,True,행복한
335,휘성,일년이면,0.665485,0.473706,0.376300,0.317116,0.110613,44.061213,46.330908,True,울고싶은
